<a href="https://colab.research.google.com/github/feniltailor22/Natural-Language-Processing/blob/main/Sentiment_Analysis_using_Transformers_and_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers 

     |████████████████████████████████| 3.1 MB 13.7 MB/s 
     |████████████████████████████████| 596 kB 87.0 MB/s 
     |████████████████████████████████| 3.3 MB 56.1 MB/s 
     |████████████████████████████████| 61 kB 410 kB/s 
     |████████████████████████████████| 895 kB 75.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


**Transformers library downloads pretrained models for Natural Language Understanding (NLU) tasks, such as analyzing the sentiment of a text, and Natural Language Generation (NLG), such as completing a prompt with new text or translating in another language.**

The easiest way to use a pretrained model on a given task is to use `pipeline`.Transformers provides the following tasks out of the box:

- Sentiment analysis: is a text positive or negative?
- Text generation (in English): provide a prompt and the model will generate what follows.
- Name entity recognition (NER): in an input sentence, label each word with the entity it represents (person, place,
  etc.)
- Question answering: provide the model with some context and a question, extract the answer from the context.
- Filling masked text: given a text with masked words (e.g., replaced by `[MASK]`), fill the blanks.
- Summarization: generate a summary of a long text.
- Translation: translate a text in another language.
- Feature extraction: return a tensor representation of the text.

In [ ]:
from transformers import pipeline

In [ ]:
classifier= pipeline(task='sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [ ]:
classifier('We are very happy to show you the 🤗 Transformers library.')

[{'label': 'POSITIVE', 'score': 0.9997795224189758}]

In [ ]:
classifier('The pizza is not that great but the crust is awesome.')

[{'label': 'POSITIVE', 'score': 0.9998461008071899}]

In [ ]:
classifier('The food was awful')

[{'label': 'NEGATIVE', 'score': 0.9995750784873962}]

**We can use it on a list of sentences, which will be preprocessed then fed to the model as a batch, returning a list of dictionaries like this one.**

In [ ]:
results = classifier(["We are very happy to show you the 🤗 Transformers library.",
           "We hope you don't hate it."])
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


label: POSITIVE, with score: 0.9998
label: NEGATIVE, with score: 0.5309


We can see the second sentence has been classified as negative (it needs to be positive or negative) but its score is
fairly neutral.

By default, the model downloaded for this pipeline is called "distilbert-base-uncased-finetuned-sst-2-english". We can
look at its [model page](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) to get more
information about it. It uses the [DistilBERT architecture](https://huggingface.co/transformers/model_doc/distilbert.html) and has been fine-tuned on a
dataset called SST-2 for the sentiment analysis task.

Let's say we want to use another model; for instance, one that has been trained on French data. We can search through
the [model hub](https://huggingface.co/models) that gathers models pretrained on a lot of data by research labs, but
also community models (usually fine-tuned versions of those big models on a specific dataset). Applying the tags
"French" and "text-classification" gives back a suggestion "nlptown/bert-base-multilingual-uncased-sentiment". Let's
see how we can use it.

You can directly pass the name of the model to use to `pipeline`:

In [ ]:
classifier= pipeline(task='sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
classifier('The food was awful') #englidh translation score

[{'label': '1 star', 'score': 0.7129524350166321}]

In [ ]:
classifier('La nourriture était horrible') #french translation score

[{'label': '1 star', 'score': 0.6887331008911133}]

In [ ]:
classifier('La comida estaba horrible') #spanish translation score

[{'label': '1 star', 'score': 0.6870307326316833}]

In [ ]:
classifier('Il cibo era terribile?') #italian translation score

[{'label': '1 star', 'score': 0.5341202020645142}]

This classifier can now deal with texts in English, French, but also Dutch, German, Italian and Spanish! You can also
replace that name by a local folder where you have saved a pretrained model (see below). You can also pass a model
object and its associated tokenizer.

We will need two classes for this. The first is `AutoTokenizer`, which we will use to download the
tokenizer associated to the model we picked and instantiate it. The second is
`AutoModelForSequenceClassification` (or
`TFAutoModelForSequenceClassification` if you are using TensorFlow), which we will use to download
the model itself. Note that if we were using the library on an other task, the class of the model would change. The
[task summary](https://huggingface.co/transformers/task_summary.html) tutorial summarizes which class is used for which task.

In [ ]:
#Using above pretained model to local folder.
#Whenever we are calling Model from the Transformers family of the Huggingface library, Two things we need to call in order to use that specific model.
#One is Tokenizer to tokenize the words. Every model works with different Tokenizer.
#Second is to download the Model itself.  

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

Now, to download the models and tokenizer we found previously, we just have to use the
`AutoModelForSequenceClassification.from_pretrained` method (feel free to replace `model_name` by
any other model from the model hub):

In [ ]:
model_name= 'nlptown/bert-base-multilingual-uncased-sentiment'
# This model only exists in PyTorch, so we use the `from_pt` flag to import that model in TensorFlow.
model= TFAutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path= model_name, from_pt= True)
tokenizer= AutoTokenizer.from_pretrained(pretrained_model_name_or_path= model_name)
classifier= pipeline(task='sentiment-analysis', model=model_name, tokenizer=tokenizer)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
classifier("I am a good boy")

[{'label': '4 stars', 'score': 0.42292681336402893}]

In [ ]:
#Under The Hood

In [ ]:
#Using the Tokenizer

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
model_name= 'distilbert-base-uncased-finetuned-sst-2-english'
tf_model= TFAutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path= model_name)
tokenizer= AutoTokenizer.from_pretrained(pretrained_model_name_or_path= model_name)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_77']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We mentioned the tokenizer is responsible for the preprocessing of your texts. First, it will split a given text in
words (or part of words, punctuation symbols, etc.) usually called *tokens*. There are multiple rules that can govern
that process (learn more about them in the [tokenizer summary](https://huggingface.co/transformers/tokenizer_summary.html)), which is why we need
to instantiate the tokenizer using the name of the model, to make sure we use the same rules as when the model was
pretrained.

The second step is to convert those *tokens* into numbers, to be able to build a tensor out of them and feed them to
the model. To do this, the tokenizer has a *vocab*, which is the part we download when we instantiate it with the
`from_pretrained` method, since we need to use the same *vocab* as when the model was pretrained.

To apply these steps on a given text, we can just feed it to our tokenizer:

In [ ]:
inputs = tokenizer("We are very happy to show you the 🤗 Transformers library.")

This returns a dictionary string to list of ints. It contains the [ids of the tokens](https://huggingface.co/transformers/glossary.html#input-ids), as
mentioned before, but also additional arguments that will be useful to the model. Here for instance, we also have an
[attention mask](https://huggingface.co/transformers/glossary.html#attention-mask) that the model will use to have a better understanding of the
sequence:

In [ ]:
print(inputs)

{'input_ids': [101, 2057, 2024, 2200, 3407, 2000, 2265, 2017, 1996, 100, 19081, 3075, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
#If we want to change the token number than we can do it as below:

In [ ]:
tf_batch= tokenizer(
    ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors='tf'
)

In [ ]:
for key, value in tf_batch.items():
    print(f"{key}: {value.numpy().tolist()}")

input_ids: [[101, 2057, 2024, 2200, 3407, 2000, 2265, 2017, 1996, 100, 19081, 3075, 1012, 102], [101, 2057, 3246, 2017, 2123, 1005, 1056, 5223, 2009, 1012, 102, 0, 0, 0]]
attention_mask: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]]
